# Ejercicios del video https://www.youtube.com/watch?v=1N5FPhYkDiE

In [6]:
### Probando con datos
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("PruebasVarias").config(conf=conf).getOrCreate()
spark

In [7]:
%%time
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
#from pyspark.sql.functions import sum, col, desc, when, lit
from pyspark.sql.functions import *

df = spark.read.csv('./datos/Admission_Predict.csv', header=True)

df.printSchema()

df.show()
# Cambio el nombre a la columna porque da error al aplicar funciones sobre la columna:
df =df.withColumnRenamed('Serial No.','numSerie')

#Cuenta cuántos nan or null hay en cada columna:

print(df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show())


root
 |-- Serial No.: string (nullable = true)
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR : string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit : string (nullable = true)

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|

### Convierte los strings a float

In [10]:
%%time

df_new = df.select([col(c).cast('float').alias(c) for c in df.columns])
df_new.printSchema()
df_new.show()

root
 |-- numSerie: float (nullable = true)
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR : float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit : float (nullable = true)



+--------+---------+-----------+-----------------+---+----+----+--------+----------------+
|numSerie|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+--------+---------+-----------+-----------------+---+----+----+--------+----------------+
|     1.0|    337.0|      118.0|              4.0|4.5| 4.5|9.65|     1.0|            0.92|
|     2.0|    324.0|      107.0|              4.0|4.0| 4.5|8.87|     1.0|            0.76|
|     3.0|    316.0|      104.0|              3.0|3.0| 3.5| 8.0|     1.0|            0.72|
|     4.0|    322.0|      110.0|              3.0|3.5| 2.5|8.67|     1.0|             0.8|
|     5.0|    314.0|      103.0|              2.0|2.0| 3.0|8.21|     0.0|            0.65|
|     6.0|    330.0|      115.0|              5.0|4.5| 3.0|9.34|     1.0|             0.9|
|     7.0|    321.0|      109.0|              3.0|3.0| 4.0| 8.2|     1.0|            0.75|
|     8.0|    308.0|      101.0|              2.0|3.0| 4.0| 7.9|     0.0|            0.68|

In [ ]:
spark.sparkContext.stop()
print('Sacabao')